# Task 6.2 Understand Theortical Background behind Estimator

## Objective 1: Understnading Primitives

Primitives are standardized interfaces that simplify quantum algorithm development by abstracting common quantum computing tasks. Qiskit provides two main primitives:

1. **Sampler**: Samples measurement outcomes from quantum circuits
2. **Estimator**: Calculating expectation values of observables

Primitives are discussed in details in previous sections 4 and 5 notebooks 4.2, 5.1 and 5.2

## Objective 2 : Error Mitigation and Suppression

- **Error Mitigation**: Post-processing techniques to reduce errors in results
- **Error Suppression**: Techniques applied during circuit execution to prevent errors

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
# if account is already saved, this will load it
service = QiskitRuntimeService()

backend = service.least_busy(operational=True, simulator=False)

### Dynamical Decoupling

Dynamical decoupling ins an error suppression techinque that inserts pulse sequences on idling qubits, so that it cancels unwanted interactions between qubits. this technique is mainly useful in circuits that have gaps, where some qubits are idle with no operations on them.
if a circuit has many operations on it, adding dynamical decoupling may worsen the performance.

In [ ]:
# enable dynamical_decoupling
estimator = Estimator(mode=backend)
estimator.options.dynamical_decoupling.enable = True
estimator.options.dynamical_decoupling.scheduling_method = "asap"
estimator.options.dynamical_decoupling.sequence_type = "XpXm"

### Pauli twirling

Twirling is an error suppresion technique that uses randmoization to shape coherent errors in sotchastic errors. Pauli twirling is a special type of twirling that uses Pauli operations.

It is implemented by adding random Pauli gates before and after the gates that we want to suppress its errors, usually two-qubits gates 

In [5]:
# enable twirling
estimator = Estimator(mode=backend)
estimator.options.twirling.enable_gates = True
estimator.options.twirling.num_randomizations = 32
estimator.options.twirling.shots_per_randomization = 100
estimator.options.twirling.strategy = "all"

### Twirled readout error extinction (TREX)

TREX is a measurement error mitigation technique that combines twirling with readout error correction. It works by substituting measurement gates with Pauli X gate, measurment that a classical bit flip gate, and this sequence is equivalent to a measurment without noise.

If a readout error occurs, TREX diagonalizes the readout error matrix making it easy to invert. while adding a small overhead on the circuit since it requires adding additional calibration circuits

In [6]:
# enable TREX
estimator = Estimator(mode=backend)
estimator.options.resilience.measure_mitigation = True
estimator.options.resilience.measure_noise_learning.num_randomizations = 32
estimator.options.resilience.measure_noise_learning.shots_per_randomization = 100

### Zero-noise extrapolation (ZNE)

Zero-Noise Extrapolation (ZNE) is an error mitigation technique that estimates the noise-free result by running the circuit at artificially increased noise levels. the results are then measured at different noise amplification factors, then the ideal reasults are estimated by extrapolating back to the zero-noise limit using mathematical models

Common noise amplification methods include gate folding and PEA (Probabilistic Error Amplification). The extrapolation can use linear, exponential, or polynomial fitting.

In [ ]:
# enable ZNE
estimator = Estimator(mode=backend)
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = "exponential"

### Probabilistic Error Amplification (PEA)

Probabilistic Error Amplification (PEA) is a noise amplification method used in Zero-Noise Extrapolation (ZNE) that is more accurate than gate folding. It consists of three stages

1. Learning: The twirled noise model of each layer of entangling gates in the circuit is learned using ` LayerNoiseLearningOptions`.
2. Noise amplification: The circuit is executed multiple times at different noise factors.
3. Extrapolation: The ideal result is estimated by extrapolating the noisy expectation value results to the zero-noise limit.

PEA is a noise amplification methods available for ZNE, so ZNE must be enabled in order to use PEA.

In [7]:
# enable PEA
estimator = Estimator(mode=backend)
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.amplifier = "pea"

### Probabilistic error cancellation (PEC)

Probabilistic Error Cancellation (PEC) is an error mitigation technique that can be used while estimating expectation values of observeables. this is done by expressing the target circuit  as a linear combination of noisy circuits. they form a quasi probability distribution, and then by combining all noisy expectation values, the ideal result can be computed. 

PEC is computationally expensive as the sampling overhaed grows exponentially with the depth of the circuit.
`max_overhead` can be used to limit the overhead

In [ ]:
# enable PEC
estimator = Estimator(mode=backend)
estimator.options.resilience.pec_mitigation = True
estimator.options.resilience.pec.max_overhead = 100

---

## Practice Questions

**1) Which option is an example of error suppression, not error mitigation?**

A) Zero-noise extrapolation (ZNE)

B) Measurement error mitigation

C) Dynamical decoupling

D) Probabilistic error cancellation


**Answer:**
<details> <br/>
C) Dynamical decoupling as it reduces error during execution


</details>

---

**2) Which circuit would benefit more from dynamical decoupling and why?**

```
# Circuit A:
qc_a = QuantumCircuit(5)
qc_a.h(0)
qc_a.cx(0, 1)
# Qubits 2, 3, 4 remain idle
qc_a.measure_all()

# Circuit B:
qc_b = QuantumCircuit(5)
qc_b.h(range(5))
for i in range(4):
    qc_b.cx(i, i+1)
qc_b.barrier()
for i in range(4, 0, -1):
    qc_b.cx(i, i-1)
qc_b.measure_all()
```

A) Both equally, as all circuits benefit from dynamical decoupling

B) Circuit A, because it has idle qubits where pulse sequences can be inserted

C) Circuit B, because dense circuits need more error suppression

D) Neither, as dynamical decoupling only works on simulators

**Answer:**
<details> <br/>
B) Circuit A

Dyanamical decoupling is more useful with circuits that has idle qubits


</details>

---

**3) Which configuration is correct for using PEA?**

```
A) estimator.options.resilience.pec_mitigation = True
   estimator.options.resilience.pec.amplifier = "pea"

B) estimator.options.resilience.zne_mitigation = True
   estimator.options.resilience.zne.amplifier = "pea"

C) estimator.options.zne.enable_gates = True
   estimator.options.zne.amplifier = "pea"

D) estimator.options.dynamical_decoupling.enable = True
   estimator.options.dynamical_decoupling.amplifier = "pea"
```

**Answer:**
<details> <br/>
B) use zne_mitigation = True

PEA is a noise amplification method for ZNE, so ZNE must be enabled.and the correct syntax to enable is `estimator.options.resilience.zne_mitigation = True`

</details>

----

**4) Why does Probabilistic Error Cancellation (PEC) produce a greater overhead on the circuit time?**

A) Because it requires quantum error correction codes that need many physical qubits

B) Because it requires to run the circuit in session mode

C) Because it needs to run the circuitw millions of times to get accurate statistics

D) Because the sampling overhead grows exponentially with circuit depth

**Answer:**
<details> <br/>

D) Because the sampling overhead grows exponentially with circuit depth

PEC expresses the target circuit as a linear combination of noisy circuits forming a quasi-probability distribution. The overhead grows exponentially with the circuit depth


</details>

---